<p align="center">
    <img src="https://upload.wikimedia.org/wikipedia/commons/7/74/Logo_%C3%89cole_normale_sup%C3%A9rieure_-_PSL_%28ENS-PSL%29.svg"
             alt="ENS-PSL"
             width="500"
             style="margin-right: 30px; display: inline-block; vertical-align: middle;"/>
    <img src="https://upload.wikimedia.org/wikipedia/commons/2/21/Caa-com_rvb.png?20210118121757"
             alt="Crédit Agricole Assurances"
             width="260"
             style="display: inline-block; vertical-align: middle;"/>
</p>

# Crédit Agricole Assurances - Fire Pure Premium Prediction
**Modélisation de la prime pure incendie pour le contrat Multirisque Agricole**

## Data Challenge 
**Powered by ENS** 

<h3><span style="color:#800000;"><strong>Authored by:</strong> <em>Alexandre Mathias DONNAT, Sr</em></span></h3>

**Curently ranked 28/253** on *https://challengedata.ens.fr/challenges/161*


This notebook presents a modern machine-learning approach to modeling fire risk for the Multirisque Agricole insurance contract, managed by Pacifica (Crédit Agricole Assurances).
The goal is to predict the **pure fire premium** "CHARGE", i.e. the expected loss (or charge) generated by fire events.
The pure premium is defined as:

$$
\text{CHARGE} = \text{FREQ} \times \text{CM} \times \text{ANNEE\_ASSURANCE}
$$

where:
- **FREQ** = expected frequency of fire claims
- **CM** = average cost per claim  
- **ANNEE_ASSURANCE** = exposure duration (years insured, provided)

##  Diving into the modeling problem

Fire events in agricultural environments are **rare**, meaning that in most years and for most contracts, the expected charge is zero.
However, when a fire does occur, it can be **extremely expensive**, due to:
- total loss of buildings or equipment,
- destruction of stock and production infrastructure,
- high reconstruction costs.

This makes the prediction problem **highly imbalanced**: rare but massive losses must be anticipated from heterogeneous contract features.
The challenge is therefore to estimate a **fair and stable premium** based on the risk profile of each contract.

## Business context

The **Multirisque Agricole** contract covers agricultural buildings, equipment, stored materials, and protects the farmer against financial and legal consequences of fire.
Fire is one of the most severe perils, both in frequency and in cost impact.
Better prediction of fire risk allows Crédit Agricole Assurances to:
- set premiums more accurately,
- identify high-risk profiles,
- support long-term financial stability of agricultural activity.

## Description of the data

Three main CSV files are provided:

#### 1) `x_train.csv` – Contract features (input variables)

This dataset contains hundreds of variables describing each contract, where uniqueness is guaranteed by an ID. For example:

- risk categories (`RISK1`, `RISK2`, …)
- geographical and meteorological information (`ZONE`, `ESPINSEE`, climate variables)
- building surfaces (`surface1`, `surface2`, …)
- insured capital amounts (`capital1`, `capital2`, …)
- prevention equipment (`prev1`, `prev2`, …)
- type of activity (crop farming, livestock, mixed production…)
- number of buildings, employees
- prior declared losses
- geographical and meteorological information

These variables are anonymized but preserve their predictive meaning.

This file is used to learn how contract characteristics influence fire risk.




In [21]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)

df = pd.read_csv('x_train.csv')
df.head()

,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,CARACT5,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG8,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,CA1,CA2,CA3,KAPITAL1,KAPITAL2,KAPITAL3,KAPITAL4,KAPITAL5,KAPITAL6,KAPITAL7,KAPITAL8,KAPITAL9,KAPITAL10,KAPITAL11,KAPITAL12,KAPITAL13,KAPITAL14,KAPITAL15,KAPITAL16,KAPITAL17,KAPITAL18,KAPITAL19,KAPITAL20,KAPITAL21,KAPITAL22,KAPITAL23,KAPITAL24,KAPITAL25,KAPITAL26,KAPITAL27,KAPITAL28,KAPITAL29,KAPITAL30,KAPITAL31,KAPITAL32,KAPITAL33,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE2,SURFACE3,SURFACE4,SURFACE5,SURFACE6,SURFACE7,SURFACE8,SURFACE9,SURFACE10,SURFACE11,SURFACE12,SURFACE13,SURFACE14,SURFACE15,SURFACE16,SURFACE17,SURFACE18,SURFACE19,SURFACE20,SURFACE21,NBBAT1,NBBAT2,NBBAT3,NBBAT4,NBBAT5,NBBAT6,NBBAT7,NBBAT8,NBBAT9,NBBAT10,NBBAT11,NBBAT13,NBBAT14,TAILLE3,TAILLE4,NBSINCONJ,NBSINSTRT,COEFASS,RISK1,RISK2,RISK3,RISK4,RISK5,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT1,EQUIPEMENT2,EQUIPEMENT3,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,EQUIPEMENT7,DISTANCE_111,DISTANCE_112,DISTANCE_121,DISTANCE_122,DISTANCE_123,DISTANCE_124,DISTANCE_131,DISTANCE_132,DISTANCE_133,...,IND_INC,IND_SNV,MEN_SURF,DISTANCE_1,DISTANCE_2,ALTITUDE_1,ALTITUDE_2,ALTITUDE_3,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,HAUTEUR,HAUTEUR_MAX,BDTOPO_BAT_MAX_HAUTEUR,ZONE_VENT,NB_CASERNES,NBJTX25_MM_A,NBJTX25_MMAX_A,NBJTX25_MSOM_A,NBJTX0_MM_A,NBJTX0_MMAX_A,NBJTX0_MSOM_A,NBJTXI27_MM_A,NBJTXI27_MMAX_A,NBJTXI27_MSOM_A,NBJTXS32_MM_A,NBJTXS32_MMAX_A,NBJTXS32_MSOM_A,NBJTXI20_MM_A,NBJTXI20_MMAX_A,NBJTXI20_MSOM_A,NBJTX30_MM_A,NBJTX30_MMAX_A,NBJTX30_MSOM_A,NBJTX35_MM_A,NBJTX35_MMAX_A,NBJTX35_MSOM_A,NBJTN10_MM_A,NBJTN10_MMAX_A,NBJTN10_MSOM_A,NBJTNI10_MM_A,NBJTNI10_MMAX_A,NBJTNI10_MSOM_A,NBJTN5_MM_A,NBJTN5_MMAX_A,NBJTN5_MSOM_A,NBJTNS25_MM_A,NBJTNS25_MMAX_A,NBJTNS25_MSOM_A,NBJTNI15_MM_A,NBJTNI15_MMAX_A,NBJTNI15_MSOM_A,NBJTNI20_MM_A,NBJTNI20_MMAX_A,NBJTNI20_MSOM_A,NBJTNS20_MM_A,NBJTNS20_MMAX_A,NBJTNS20_MSOM_A,NBJTMS24_MM_A,NBJTMS24_MMAX_A,NBJTMS24_MSOM_A,TAMPLIAB_VOR_MM_A,TAMPLIAB_VOR_MMAX_A,TAMPLIM_VOR_MM_A,TAMPLIM_VOR_MMAX_A,TM_VOR_MM_A,TM_VOR_MMAX_A,TMM_VOR_MM_A,TMM_VOR_MMAX_A,TMMAX_VOR_MM_A,TMMAX_VOR_MMAX_A,TMMIN_VOR_MM_A,TMMIN_VOR_MMAX_A,TN_VOR_MM_A,TN_VOR_MMAX_A,TNAB_VOR_MM_A,TNAB_VOR_MMAX_A,TNMAX_VOR_MM_A,TNMAX_VOR_MMAX_A,TX_VOR_MM_A,TX_VOR_MMAX_A,TXAB_VOR_MM_A,TXAB_VOR_MMAX_A,TXMIN_VOR_MM_A,TXMIN_VOR_MMAX_A,NBJFF10_MM_A,NBJFF10_MMAX_A,NBJFF10_MSOM_A,NBJFF16_MM_A,NBJFF16_MMAX_A,NBJFF16_MSOM_A,NBJFF28_MM_A,NBJFF28_MMAX_A,NBJFF28_MSOM_A,NBJFXI3S10_MM_A,NBJFXI3S10_MMAX_A,NBJFXI3S10_MSOM_A,NBJFXI3S16_MM_A,NBJFXI3S16_MMAX_A,NBJFXI3S16_MSOM_A,NBJFXI3S28_MM_A,NBJFXI3S28_MMAX_A,NBJFXI3S28_MSOM_A,NBJFXY8_MM_A,NBJFXY8_MMAX_A,NBJFXY8_MSOM_A,NBJFXY10_MM_A,NBJFXY10_MMAX_A,NBJFXY10_MSOM_A,NBJFXY15_MM_A,NBJFXY15_MMAX_A,NBJFXY15_MSOM_A,FFM_VOR_MM_A,FFM_VOR_MMAX_A,FXI3SAB_VOR_MM_A,FXI3SAB_VOR_MMAX_A,FXIAB_VOR_MM_A,FXIAB_VOR_MMAX_A,FXYAB_VOR_MM_A,FXYAB_VOR_MMAX_A,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,FXI3SAB_VOR_COM_MMAX_A_Y,NBJRR50_MM_A,NBJRR50_MMAX_A,NBJRR50_MSOM_A,NBJRR1_MM_A,NBJRR1_MMAX_A,NBJRR1_MSOM_A,NBJRR5_MM_A,NBJRR5_MMAX_A,NBJRR5_MSOM_A,NBJRR10_MM_A,NBJRR10_MMAX_A,NBJRR10_MSOM_A,NBJRR30_MM_A,NBJRR30_MMAX_A,NBJRR30_MSOM_A,NBJRR100_MM_A,NBJRR100_MMAX_A,NBJRR100_MSOM_A,RR_VOR_MM_A,RR_VOR_MMAX_A,RRAB_VOR_MM_A,RRAB_VOR_MMAX_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,1,ACT1,VOC6,1,0,N,N,NaN,NaN,N,NaN,absence de surface,0,NaN,CLASS5,0,2,NaN,N,N,N,N,N,N,N,N,N,N,N,NaN,NaN,NaN,100.0,NaN,05 - [1M - 1.5M],05 - [750k- 1M],0,0,0,0,0,0,0,0,1,0,0,0,0,0.0,125000,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,1000,0,0,0,0.0,0.0,0.0,125000,0.0,N,N,N,N,N,N,O,N,N,N,1600,1600,NaN,1500,0,1500,50.0,NaN,0.0,50.0,NaN,0,0,0,0,0,0,NaN,NaN,0,0,9,0,0,8,0,0,0,0,3,0,0,8,0,2,0,0.0,0.00,01-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,O,

####  2) `y_train.csv` – True outputs (targets)

For each row in `x_train`, the file provides the observed values:
- **FREQ** — fire claim frequency
- **CM** — average cost of claims
- **CHARGE** — total loss = FREQ × CM × ANNEE
- **ANNEE_ASSURANCE** — exposure duration

This is the "ground truth" the model must learn to reproduce.



In [5]:
df = pd.read_csv('y_train.csv')
df.head()

,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


#### 3) `x_test.csv` – New contracts to predict

Same structure as `x_train`, but without target variables.

Our task is to produce a file `submission.csv` containing:
- predicted **FREQ**
- predicted **CM**
- predicted **CHARGE**
- predicted **ANNEE_ASSURANCE** (actually given)

##  How the scoring works (black-box evaluation)

We do not have access to the true `y_test`.

When we upload our predictions to the challenge platform:
1. The platform compares our predictions to the true hidden test values.
2. It computes the error using the **RMSE** metric:

$$
\text{RMSE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

This error becomes our public leaderboard score.
This means that **model selection must be done using internal validation**, not using the hidden test labels.

## The problem to solve

### 1. Predicting FREQ (fire frequency)

**FREQ** represents the number of fire events per year of exposure.  
Because fires are rare, FREQ is usually very small.

Fire occurrence is naturally modeled with a **Poisson distribution**.

#### Reminder: Poisson distribution

- **Parameter:** $\lambda > 0$
- **Probability of observing $k$ events:**

$$
P(N = k) = \frac{\lambda^k e^{-\lambda}}{k!}
$$

Poisson models:
- discrete event counts,
- independent occurrences,
- over an exposure interval,
- with mean intensity $\lambda$.

Here,

$$
\lambda = \text{expected frequency} \times \text{exposure}
$$

### 2. Predicting CM (average claim cost)

Claim amounts are highly variable, right-skewed, and contain extreme values.  
To stabilize them, we model:

$$
\log(1 + \text{CM})
$$

### 3. Predicting CHARGE

Two strategies exist:

- **Structural:**  
$$
\text{CHARGE} = \text{FREQ} \times \text{CM} \times \text{ANNEE\_ASSURANCE}
$$

- **Direct:**  
Predict CHARGE directly from features using a ML model.

Our pipeline combines both approaches.

## Our method pipeline 

We combine **actuarial reasoning** and **modern machine learning**.

### I - Preprocessing
- missing-value imputation
- ordinal encoding
- log-transformations
- feature engineering (aggregations)
- train/validation split

### II - Three LightGBM models
- **LightGBM Poisson** for FREQ
- **LightGBM log-regression** for CM
- **LightGBM direct model** on log(CHARGE)

### III - Blending and calibration (on validation)

On the validation set, we build two estimators of CHARGE:

- structural reconstruction  
$$
\text{CHARGE}_{\text{mult}} = \widehat{\text{FREQ}} \times \widehat{\text{CM}} \times \text{ANNEE\_ASSURANCE}
$$
- direct prediction from the CHARGE model  
$$
\text{CHARGE}_{\text{direct}}
$$

We then:
1. search for the best blending weight $w$ such that  
$$
\text{CHARGE}_{\text{blend}} = w \cdot \text{CHARGE}_{\text{mult}} + (1 - w) \cdot \text{CHARGE}_{\text{direct}},
$$
2. fit a linear calibration  
$$
\text{CHARGE}_{\text{final}} = a + b \cdot \text{CHARGE}_{\text{blend}}
$$
to correct global bias and further reduce RMSE.
Both $w$, $a$ and $b$ are learned **only on the validation fold**.

### IV - Full retraining on the whole training set

Using the optimal number of iterations found on validation, we:

- re-encode all features on the full `X_train`,  
- refit the three LightGBM models (FREQ, CM, CHARGE) on the entire training data.
No blending or calibration is done here; this step only rebuilds the final models on all available data.

### V - Test predictions and submission file

On `x_test.csv`, we:

- generate predictions for **FREQ**, **CM**, and direct **CHARGE**,  
- rebuild  
$$
\text{CHARGE}_{\text{mult}} = \widehat{\text{FREQ}} \times \widehat{\text{CM}} \times \text{ANNEE\_ASSURANCE},
$$
- apply the **same blending weight $w$** and **calibration parameters $a, b$** learned on validation to obtain  
$$
\text{CHARGE}_{\text{final}},
$$
- run sanity checks (no NaN/Inf, reasonable ranges),
- export the final `submission.csv` file for the competition platform.

## Futures improvments

Our final blended + calibrated model achieves an RMSE of **5600.57** on the public leaderboard.  
This score reflects several rounds of hyperparameter refinement and stability checks.  
It is a solid performance for this challenge, and in the final section we outline several potential improvements, from safe parameter tuning to more complex ensemble strategies, together with the risks associated with each approach.


Several paths could reduce the final RMSE:

### 1. Hyperparameter optimization (safe, low-risk)
- Reduce/Increase `num_leaves`
- Tune `min_child_samples` (controls overfitting)
- Test `learning_rate ∈ {0.01, 0.02, 0.05}`
- Wider search for `reg_lambda` to control extreme CM values  
These adjustments are simple and rarely break the model.

### 2. Better preprocessing
- Target encoding for categorical variables  
- Group rare categories more intelligently  
- Add engineered features (capital density, buildings × equipment, geographic risk interactions)

### 3. Model upgrades
- Gradient-boosted Poisson for FREQ (e.g. CatBoost)
- Tweedie regression for CHARGE (built-in in LightGBM)
- Quantile regression to stabilise extreme costs

### 4. Ensemble learning (medium complexity)
- Average predictions from LightGBM + CatBoost + XGBoost  
- Stacking meta-models on top of FREQ/CM/CHARGE  
Ensembles often give +1–3% RMSE reduction.

### 5. More advanced methods (higher risk)
- Neural networks for tabular data (TabNet / FT-Transformer)  
  → require heavy tuning, risk of overfitting  
- Survival models for severity estimation  
- Hierarchical GLM + ML hybrid models (actuarial + ML)

### 6. Additional validation strategies
- K-fold cross validation  
- Time-based folds using ANNEE_ASSURANCE  

These provide more stable estimates of generalisation error.

# I.1. Modules

In [29]:
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", 300)

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error

from category_encoders import OrdinalEncoder
import lightgbm as lgb

# I.2. Data upload

In [11]:
X_train = pd.read_csv("x_train.csv")
y_train = pd.read_csv("y_train.csv")
X_test  = pd.read_csv("x_test.csv")

print("Shapes :")
print("X_train :", X_train.shape)
print("y_train :", y_train.shape)
print("X_test  :", X_test.shape)

Shapes :
X_train : (383610, 374)
y_train : (383610, 5)
X_test  : (95852, 374)


# I.3. Target variables

Here we extract the three targets (FREQ, CM, CHARGE) and the exposure ANNEE_ASSURANCE, which will be used both in the loss definition and later for multiplicative reconstruction of CHARGE.


In [12]:
# Target variables
y_freq   = y_train["FREQ"]
y_cm     = y_train["CM"]
y_charge = y_train["CHARGE"]
y_annee  = y_train["ANNEE_ASSURANCE"].values  # for the formula
# We do not keep ANNEE_ASSURANCE as a feature (already in the definition of targets)

# I.4. Simple feature engineering

We create a few aggregated features (e.g., total capital, surface, number of buildings) by summing related columns. These give the model more stable signals. We also add basic log-transforms to reduce the impact of large numerical values.


In [13]:
def add_sum_feature(df, prefix, new_name):
    # Columns that start with the prefix
    cols = [c for c in df.columns if c.startswith(prefix)]
    # Keep those that are numeric
    num_cols = [c for c in cols if pd.api.types.is_numeric_dtype(df[c])]
    if num_cols:
        df[new_name] = df[num_cols].fillna(0).sum(axis=1)
        print(f"{new_name} créé à partir de {len(num_cols)} colonnes ({prefix})")
    else:
        print(f"Aucune colonne numérique pour le préfixe {prefix}")
    return num_cols

for df in (X_train, X_test):
    add_sum_feature(df, "KAPITAL",  "KAPITAL_TOTAL")
    add_sum_feature(df, "SURFACE",  "SURFACE_TOTAL")
    add_sum_feature(df, "NBBAT",    "NBBAT_TOTAL")
    add_sum_feature(df, "RISK",     "RISK_NB")
    add_sum_feature(df, "EQUIPEMENT", "EQUIPEMENT_NB")

# Some simple log-transforms (robust with log1p)
for df in (X_train, X_test):
    for col in ["KAPITAL_TOTAL", "SURFACE_TOTAL", "NBBAT_TOTAL"]:
        if col in df.columns:
            df[f"log1p_{col}"] = np.log1p(df[col].fillna(0))

KAPITAL_TOTAL créé à partir de 33 colonnes (KAPITAL)
SURFACE_TOTAL créé à partir de 19 colonnes (SURFACE)
NBBAT_TOTAL créé à partir de 13 colonnes (NBBAT)
RISK_NB créé à partir de 6 colonnes (RISK)
EQUIPEMENT_NB créé à partir de 5 colonnes (EQUIPEMENT)
KAPITAL_TOTAL créé à partir de 33 colonnes (KAPITAL)
SURFACE_TOTAL créé à partir de 19 colonnes (SURFACE)
NBBAT_TOTAL créé à partir de 13 colonnes (NBBAT)
RISK_NB créé à partir de 6 colonnes (RISK)
EQUIPEMENT_NB créé à partir de 5 colonnes (EQUIPEMENT)


# I.5. Identify numerical and categorical features

We split the dataset into numerical and categorical columns, excluding `ID` and `ANNEE_ASSURANCE` from the feature set. This separation prepares the preprocessing and encoding steps.

In [28]:
# Identify numeric columns
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# We do not want ID / ANNEE_ASSURANCE as raw features
for col_to_drop in ["ID", "ANNEE_ASSURANCE"]:
    if col_to_drop in numeric_cols:
        numeric_cols.remove(col_to_drop)

# Remaining = categorical
categorical_cols = [c for c in X_train.columns if c not in numeric_cols]

print("Nb numeric :", len(numeric_cols))
print("Nb categorical :", len(categorical_cols))


Nb numeric : 100
Nb categorical : 282


# I.6. Basic preprocessing and train/validation split

We apply simple imputations (median for numerical features, "MISSING" for categorical ones) to ensure clean inputs.  
We then create a train/validation split to evaluate the models internally, since the real test targets are hidden.


In [20]:
# Input simple for numeric and categorical columns
X_train_proc = X_train.copy()
X_test_proc = X_test.copy()

# Numeric imputation: median
for col in numeric_cols:
    median_val = X_train_proc[col].median()
    X_train_proc[col].fillna(median_val, inplace=True)
    X_test_proc[col].fillna(median_val, inplace=True)

# Categorical input: 'MISSING'
for col in categorical_cols:
    if col in X_train_proc.columns:
        X_train_proc[col].fillna('MISSING', inplace=True)
        X_test_proc[col].fillna('MISSING', inplace=True)

print("Preprocessing done.")

# We keep ANNEE_ASSURANCE from y_train to calculate CHARGE on validation
annee_all = y_annee  # already a np.array above

X_train_raw, X_valid_raw, \
y_freq_train, y_freq_valid, \
y_cm_train,   y_cm_valid, \
y_charge_train, y_charge_valid, \
annee_train,  annee_valid = train_test_split(
    X_train_proc,
    y_freq,
    y_cm,
    y_charge,
    annee_all,
    test_size=0.2,
    random_state=42
)

print("Split shapes :")
print("X_train_raw :", X_train_raw.shape)
print("X_valid_raw :", X_valid_raw.shape)

Preprocessing done.
Split shapes :
X_train_raw : (306888, 382)
X_valid_raw : (76722, 382)


# I.7. Ordinal encoding of categorical variables

We remove `ID` and `ANNEE_ASSURANCE` from the feature set, then apply an ordinal encoding to all categorical columns. This turns categories into integers in a consistent way between train, validation and test.


In [22]:
# Features columns (we remove ID + ANNEE_ASSURANCE)
cols_features = [c for c in X_train_proc.columns if c not in ["ID", "ANNEE_ASSURANCE"]]

X_train_raw = X_train_raw[cols_features].copy()
X_valid_raw = X_valid_raw[cols_features].copy()
X_test_raw  = X_test_proc[cols_features].copy()

# Categorial columns truly present in the features
cat_cols_enc = [c for c in categorical_cols if c in cols_features]
print("Nb colonnes catégorielles encodées :", len(cat_cols_enc))

encoder = OrdinalEncoder(
    cols=cat_cols_enc,
    handle_unknown="impute",
    handle_missing="impute"
)

encoder.fit(X_train_raw)

X_train_enc = encoder.transform(X_train_raw)
X_valid_enc = encoder.transform(X_valid_raw)
X_test_enc  = encoder.transform(X_test_raw)

X_train_enc.head()

Nb colonnes catégorielles encodées : 280


,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,DUREE_REQANEUF,CARACT4,CARACT5,TYPBAT1,INDEM2,TYPBAT2,FRCH1,FRCH2,DEROG1,DEROG2,DEROG3,DEROG4,DEROG5,DEROG6,DEROG7,DEROG8,DEROG9,DEROG10,DEROG11,DEROG12,DEROG13,DEROG14,DEROG15,DEROG16,TAILLE1,TAILLE2,CA1,CA2,CA3,KAPITAL1,KAPITAL2,KAPITAL3,KAPITAL4,KAPITAL5,KAPITAL6,KAPITAL7,KAPITAL8,KAPITAL9,KAPITAL10,KAPITAL11,KAPITAL12,KAPITAL13,KAPITAL14,KAPITAL15,KAPITAL16,KAPITAL17,KAPITAL18,KAPITAL19,KAPITAL20,KAPITAL21,KAPITAL22,KAPITAL23,KAPITAL24,KAPITAL25,KAPITAL26,KAPITAL27,KAPITAL28,KAPITAL29,KAPITAL30,KAPITAL31,KAPITAL32,KAPITAL33,KAPITAL34,KAPITAL35,KAPITAL36,KAPITAL37,KAPITAL38,KAPITAL39,KAPITAL40,KAPITAL41,KAPITAL42,KAPITAL43,SURFACE1,SURFACE2,SURFACE3,SURFACE4,SURFACE5,SURFACE6,SURFACE7,SURFACE8,SURFACE9,SURFACE10,SURFACE11,SURFACE12,SURFACE13,SURFACE14,SURFACE15,SURFACE16,SURFACE17,SURFACE18,SURFACE19,SURFACE20,SURFACE21,NBBAT1,NBBAT2,NBBAT3,NBBAT4,NBBAT5,NBBAT6,NBBAT7,NBBAT8,NBBAT9,NBBAT10,NBBAT11,NBBAT13,NBBAT14,TAILLE3,TAILLE4,NBSINCONJ,NBSINSTRT,COEFASS,RISK1,RISK2,RISK3,RISK4,RISK5,RISK6,RISK7,RISK8,RISK9,RISK10,RISK11,RISK12,RISK13,EQUIPEMENT1,EQUIPEMENT2,EQUIPEMENT3,EQUIPEMENT4,EQUIPEMENT5,EQUIPEMENT6,EQUIPEMENT7,DISTANCE_111,DISTANCE_112,DISTANCE_121,DISTANCE_122,DISTANCE_123,DISTANCE_124,DISTANCE_131,DISTANCE_132,DISTANCE_133,DISTANCE_141,...,ALTITUDE_3,ALTITUDE_4,ALTITUDE_5,BDTOPO_BAT_MAX_HAUTEUR_MAX,HAUTEUR,HAUTEUR_MAX,BDTOPO_BAT_MAX_HAUTEUR,ZONE_VENT,NB_CASERNES,NBJTX25_MM_A,NBJTX25_MMAX_A,NBJTX25_MSOM_A,NBJTX0_MM_A,NBJTX0_MMAX_A,NBJTX0_MSOM_A,NBJTXI27_MM_A,NBJTXI27_MMAX_A,NBJTXI27_MSOM_A,NBJTXS32_MM_A,NBJTXS32_MMAX_A,NBJTXS32_MSOM_A,NBJTXI20_MM_A,NBJTXI20_MMAX_A,NBJTXI20_MSOM_A,NBJTX30_MM_A,NBJTX30_MMAX_A,NBJTX30_MSOM_A,NBJTX35_MM_A,NBJTX35_MMAX_A,NBJTX35_MSOM_A,NBJTN10_MM_A,NBJTN10_MMAX_A,NBJTN10_MSOM_A,NBJTNI10_MM_A,NBJTNI10_MMAX_A,NBJTNI10_MSOM_A,NBJTN5_MM_A,NBJTN5_MMAX_A,NBJTN5_MSOM_A,NBJTNS25_MM_A,NBJTNS25_MMAX_A,NBJTNS25_MSOM_A,NBJTNI15_MM_A,NBJTNI15_MMAX_A,NBJTNI15_MSOM_A,NBJTNI20_MM_A,NBJTNI20_MMAX_A,NBJTNI20_MSOM_A,NBJTNS20_MM_A,NBJTNS20_MMAX_A,NBJTNS20_MSOM_A,NBJTMS24_MM_A,NBJTMS24_MMAX_A,NBJTMS24_MSOM_A,TAMPLIAB_VOR_MM_A,TAMPLIAB_VOR_MMAX_A,TAMPLIM_VOR_MM_A,TAMPLIM_VOR_MMAX_A,TM_VOR_MM_A,TM_VOR_MMAX_A,TMM_VOR_MM_A,TMM_VOR_MMAX_A,TMMAX_VOR_MM_A,TMMAX_VOR_MMAX_A,TMMIN_VOR_MM_A,TMMIN_VOR_MMAX_A,TN_VOR_MM_A,TN_VOR_MMAX_A,TNAB_VOR_MM_A,TNAB_VOR_MMAX_A,TNMAX_VOR_MM_A,TNMAX_VOR_MMAX_A,TX_VOR_MM_A,TX_VOR_MMAX_A,TXAB_VOR_MM_A,TXAB_VOR_MMAX_A,TXMIN_VOR_MM_A,TXMIN_VOR_MMAX_A,NBJFF10_MM_A,NBJFF10_MMAX_A,NBJFF10_MSOM_A,NBJFF16_MM_A,NBJFF16_MMAX_A,NBJFF16_MSOM_A,NBJFF28_MM_A,NBJFF28_MMAX_A,NBJFF28_MSOM_A,NBJFXI3S10_MM_A,NBJFXI3S10_MMAX_A,NBJFXI3S10_MSOM_A,NBJFXI3S16_MM_A,NBJFXI3S16_MMAX_A,NBJFXI3S16_MSOM_A,NBJFXI3S28_MM_A,NBJFXI3S28_MMAX_A,NBJFXI3S28_MSOM_A,NBJFXY8_MM_A,NBJFXY8_MMAX_A,NBJFXY8_MSOM_A,NBJFXY10_MM_A,NBJFXY10_MMAX_A,NBJFXY10_MSOM_A,NBJFXY15_MM_A,NBJFXY15_MMAX_A,NBJFXY15_MSOM_A,FFM_VOR_MM_A,FFM_VOR_MMAX_A,FXI3SAB_VOR_MM_A,FXI3SAB_VOR_MMAX_A,FXIAB_VOR_MM_A,FXIAB_VOR_MMAX_A,FXYAB_VOR_MM_A,FXYAB_VOR_MMAX_A,FFM_VOR_COM_MM_A_Y,FFM_VOR_COM_MMAX_A_Y,FXI3SAB_VOR_COM_MM_A_Y,FXI3SAB_VOR_COM_MMAX_A_Y,NBJRR50_MM_A,NBJRR50_MMAX_A,NBJRR50_MSOM_A,NBJRR1_MM_A,NBJRR1_MMAX_A,NBJRR1_MSOM_A,NBJRR5_MM_A,NBJRR5_MMAX_A,NBJRR5_MSOM_A,NBJRR10_MM_A,NBJRR10_MMAX_A,NBJRR10_MSOM_A,NBJRR30_MM_A,NBJRR30_MMAX_A,NBJRR30_MSOM_A,NBJRR100_MM_A,NBJRR100_MMAX_A,NBJRR100_MSOM_A,RR_VOR_MM_A,RR_VOR_MMAX_A,RRAB_VOR_MM_A,RRAB_VOR_MMAX_A,ESPINSEE,AN_EXERC,ZONE,KAPITAL_TOTAL,SURFACE_TOTAL,NBBAT_TOTAL,RISK_NB,EQUIPEMENT_NB,log1p_KAPITAL_TOTAL,log1p_SURFACE_TOTAL,log1p_NBBAT_TOTAL
225966,1,1,2,11,1,1,1.0,1,1,2.0,1,0,1,1,2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,100.0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,62500,62500.0,62500,0,0,0.0,0.0,0.0,0.0,0.0,0.0,2000.0,0.0,0.0,0.0,1000,0,0,0,0.0,0.0,0.0,125000,0.0,1,1,1,1,1,1,1,1,1,1,2700,2700,2700.0,1,0,1,50.0,0.0,0.0,100.0,2700.0,0,0,0,0,0,50,0.0,0.0,0,0,7,0,0,6,0,0,0,0,0,0,0,6,0,0,0,1.2,1.20,1,3.0,24.0,9.0,3.0,4.0,1,80.0,1,1,1,1,1,1,0,1,0,1,1,11,12,1

# II - Three LightGBM Models

### Choice of hyperparameters?

The LightGBM configurations used for FREQ, CM and CHARGE were not chosen arbitrarily.  
They follow standard actuarial ML heuristics designed for tabular, noisy and highly heterogeneous insurance data.

**Main design choices:**

- **num_leaves = 256**  
  A moderately large tree capacity.  
  Insurance interactions are complex (geography × risk level × capital × activities), so deeper trees help capture nonlinearities.

- **learning_rate = 0.03**  
  A small learning rate improves stability and reduces overfitting, especially when costs contain extreme values.

- **subsample = 0.9 & colsample_bytree = 0.9**  
  Light regularization by stochasticity.  
  Prevents the model from memorizing rare extreme claims.

- **min_child_samples = 30**  
  Avoids creating leaves supported by very few samples — crucial for rare-event modelling (fires ≈ rare).

- **reg_alpha / reg_lambda**  
  - FREQ uses **(0.1 / 0.2)** : moderate regularization for a Poisson model.  
  - CM and CHARGE use **λ = 0.5** : CM is noisier with heavier tails → stronger shrinkage improves robustness.

- **n_estimators = 4000 with early stopping**  
  Standard gradient boosting trick: allow many trees but automatically stop early.  
  This gives the model enough capacity without overfitting.

- **objective = poisson / regression**  
  Perfectly aligned with:
  - count modelling for **FREQ**,  
  - continuous loss modelling for **CM** and **CHARGE**.

# II.1. LightGBM Model for FREQ (Poisson)

We train a LightGBM model with a Poisson objective, which is well-suited for rare event counts.  
Before reaching this final setup, we experimented with smaller trees (`num_leaves=156`) and lower subsampling (`subsample=0.8`), but the configuration below provided a better bias–variance balance on validation.


In [23]:
params_freq = {
    "objective": "poisson",
    "metric": "rmse",
    "learning_rate": 0.03,
    "num_leaves": 256,            
    "max_depth": -1,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "min_child_samples": 30,
    "reg_alpha": 0.1,
    "reg_lambda": 0.2,
    "n_estimators": 4000,
    "n_jobs": -1,
    "random_state": 42,
}

lgb_freq = lgb.LGBMRegressor(**params_freq)

lgb_freq.fit(
    X_train_enc, y_freq_train,
    eval_set=[(X_valid_enc, y_freq_valid)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(stopping_rounds=300), lgb.log_evaluation(200)]
)

best_iter_freq = lgb_freq.best_iteration_
print("Best iter FREQ :", best_iter_freq)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.333206 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4600
[LightGBM] [Info] Number of data points in the train set: 306888, number of used features: 376
[LightGBM] [Info] Start training from score -4.417608
Training until validation scores don't improve for 300 rounds
[200]	valid_0's rmse: 0.688952
Early stopping, best iteration is:
[9]	valid_0's rmse: 0.685193
Best iter FREQ : 9


# II.2. LightGBM Model for CM (on log1p(CM))

We model the average claim cost using a LightGBM regression on `log1p(CM)` to stabilize the heavy right tail of claim amounts.  
Compared to the frequency model, we increase L2 regularization (`reg_lambda=0.5`) to reduce variance, as CM is more volatile and sensitive to outliers.


In [24]:
y_cm_train_log = np.log1p(y_cm_train)
y_cm_valid_log = np.log1p(y_cm_valid)

params_cm = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.03,
    "num_leaves": 256,
    "max_depth": -1,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "min_child_samples": 30,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,   # More L2 regularization to stabilize CM
    "n_estimators": 4000,
    "n_jobs": -1,
    "random_state": 43,
}

lgb_cm = lgb.LGBMRegressor(**params_cm)

lgb_cm.fit(
    X_train_enc, y_cm_train_log,
    eval_set=[(X_valid_enc, y_cm_valid_log)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(stopping_rounds=300), lgb.log_evaluation(200)]
)

best_iter_cm = lgb_cm.best_iteration_
print("Best iter CM :", best_iter_cm)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.250407 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4613
[LightGBM] [Info] Number of data points in the train set: 306888, number of used features: 378
[LightGBM] [Info] Start training from score 0.048599
Training until validation scores don't improve for 300 rounds
[200]	valid_0's rmse: 0.640911
Early stopping, best iteration is:
[30]	valid_0's rmse: 0.635737
Best iter CM : 30


**Notes:**  
The validation logs for the FREQ and CM models are not interpreted individually, because these models are only intermediate components of the final blended prediction. Their role is to provide stable estimates for the structural decomposition (CHARGE = FREQ × CM × ANNEE). Only the final blended and calibrated CHARGE model will be evaluated.


# II.3. LightGBM Model direct on CHARGE (log1p)

In addition to the structural decomposition (FREQ × CM × ANNEE), we train a direct LightGBM regression model on `log1p(CHARGE)`.  
This model acts as a global shortcut: it captures nonlinear interactions between features that the multiplicative structure cannot express.  
Its predictions will later be blended with the structural reconstruction to improve overall performance.


In [ ]:
y_charge_train_log = np.log1p(y_charge_train)
y_charge_valid_log = np.log1p(y_charge_valid)

params_charge = {
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.03,
    "num_leaves": 256,
    "max_depth": -1,
    "subsample": 0.9,
    "colsample_bytree": 0.9,
    "min_child_samples": 30,
    "reg_alpha": 0.1,
    "reg_lambda": 0.5,
    "n_estimators": 4000,
    "n_jobs": -1,
    "random_state": 44,
}

lgb_charge = lgb.LGBMRegressor(**params_charge)

lgb_charge.fit(
    X_train_enc, y_charge_train_log,
    eval_set=[(X_valid_enc, y_charge_valid_log)],
    eval_metric="rmse",
    callbacks=[lgb.early_stopping(stopping_rounds=300), lgb.log_evaluation(200)]
)

best_iter_charge = lgb_charge.best_iteration_
print("Best iter CHARGE direct :", best_iter_charge)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.274220 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4611
[LightGBM] [Info] Number of data points in the train set: 306888, number of used features: 377
[LightGBM] [Info] Start training from score 0.048766
Training until validation scores don't improve for 300 rounds
[200]	valid_0's rmse: 0.64341
Early stopping, best iteration is:
[29]	valid_0's rmse: 0.638164
Best iter CHARGE direct : 29


**Notes:**  
The standalone CHARGE model (log-transformed) is not interpreted individually. Just like FREQ and CM, it acts as one component of the final ensemble.  
Its validation RMSE only tells us how good a *direct* prediction of CHARGE can be, but this number is not used in isolation.

We remind : The important part is its contribution to the blended prediction (combined with the multiplicative reconstruction FREQ × CM × ANNEE). Only this blended + calibrated CHARGE prediction will ultimately be evaluated on the competition platform.

# III.1 Validation predictions and first comparison

We now evaluate two independent ways of reconstructing CHARGE on the validation set:

1. **Multiplicative reconstruction**  
    $$
    \widehat{\text{CHARGE}}_{\text{mult}} 
    = \widehat{\text{FREQ}} \times \widehat{\text{CM}} \times \text{ANNEE\_ASSURANCE}
    $$
    This follows the actuarial decomposition used by the insurer.

2. **Direct CHARGE model**  
    $$
    \widehat{\text{CHARGE}}_{\text{direct}} = \exp(\widehat{\log(\text{CHARGE} + 1)}) - 1
    $$

We compute both RMSE scores on the validation set.  
These raw RMSE values allow us to understand which modelling path performs better individually, but **no decision is taken yet**.


In [ ]:
# Predictions on FREQ / CM on validation
pred_freq_valid = np.maximum(
    lgb_freq.predict(X_valid_enc, num_iteration=best_iter_freq),
    0
)
pred_cm_valid_log = lgb_cm.predict(X_valid_enc, num_iteration=best_iter_cm)
pred_cm_valid = np.maximum(np.expm1(pred_cm_valid_log), 0)

# 1. CHARGE multiplicative (I)
charge_mult_valid = pred_freq_valid * pred_cm_valid * annee_valid

# 2. Prediction CHARGE direct (II) (log)
pred_charge_valid_log = lgb_charge.predict(X_valid_enc, num_iteration=best_iter_charge)
charge_direct_valid = np.maximum(np.expm1(pred_charge_valid_log), 0)

# RMSE separated
rmse_mult = np.sqrt(mean_squared_error(y_charge_valid, charge_mult_valid))
rmse_direct = np.sqrt(mean_squared_error(y_charge_valid, charge_direct_valid))

print(f"RMSE validation CHARGE multiplicatif : {rmse_mult:.4f}")
print(f"RMSE validation CHARGE direct       : {rmse_direct:.4f}")


RMSE validation CHARGE multiplicatif : 7045.2809
RMSE validation CHARGE direct       : 7045.2789


**Interpretation of the validation RMSEs**

Both modelling strategies show almost identical performance on the validation fold:
- Multiplicative reconstruction (FREQ × CM × ANNEE_ASSURANCE): **7045.2809**
- Direct CHARGE model: **7045.2789**

This was expected: the multiplicative model is theoretically well-aligned with the business formulation of the risk, while the direct model benefits from LightGBM’s flexibility.

Since neither approach clearly dominates, the best strategy is to **combine both**.  
The next step is therefore to perform a **blending search** to find the optimal mix of the two predictions.


# III.2. Blending and linear calibration on validation

We now combine the two CHARGE estimators on the validation set:

- structural reconstruction: `CHARGE_mult = FREQ × CM × ANNEE`
- direct model: `CHARGE_direct` from LightGBM

We search for the best blending weight $w$ and then apply a simple linear calibration  
$$
\text{CHARGE}_{\text{final}} = a + b \cdot \text{CHARGE}_{\text{blend}}
$$
to correct global bias.

### Why linear calibration?

Even after blending, models like LightGBM tend to produce predictions that are:

- globally too high or too low (bias),
- too compressed or too spread relative to the true values (scale error).

A simple affine correction solves it:

$$
\text{CHARGE}_{\text{final}} = a + b \cdot \text{CHARGE}_{\text{blend}}
$$

where:

- $a$ corrects the global offset (systematic under/overestimation),
- $b$ rescales predictions to match the true variability.

Both parameters $a$ and $b$ are **learned automatically on the validation fold** using covariance and variance formulas (equivalent to a univariate linear regression without intercept fit).


In [ ]:
# Blending weight search on validation
best_rmse = float('inf')
best_w = 0.5  # default

for w in np.linspace(0, 1, 21):
    charge_blend = w * charge_mult_valid + (1 - w) * charge_direct_valid
    rmse_blend = np.sqrt(mean_squared_error(y_charge_valid, charge_blend))
    if rmse_blend < best_rmse:
        best_rmse = rmse_blend
        best_w = w

print(f"Optimal blend weight (validation) : {best_w:.3f}")
print(f"RMSE validation CHARGE blend      : {best_rmse:.4f}")

# Linear calibration on validation
true = y_charge_valid.values
pred = best_w * charge_mult_valid + (1 - best_w) * charge_direct_valid

m_pred = pred.mean()
m_true = true.mean()
var_pred = pred.var()
cov = ((pred - m_pred) * (true - m_true)).mean()

if var_pred > 0:
    b = cov / var_pred
else:
    b = 1.0
a = m_true - b * m_pred

rmse_before = np.sqrt(mean_squared_error(true, pred))
rmse_after  = np.sqrt(mean_squared_error(true, np.maximum(a + b * pred, 0)))

print("Calibration parameters:")
print("  a (intercept) :", a)
print("  b (slope)     :", b)
print(f"RMSE before calibration : {rmse_before:.4f}")
print(f"RMSE after  calibration : {rmse_after:.4f}")


Optimal blend weight (validation) : 0.000
RMSE validation CHARGE blend      : 7045.2789
Calibration parameters:
  a (intercept) : 133.99884802206378
  b (slope)     : 1376.048739401628
RMSE before calibration : 7045.2789
RMSE after  calibration : 7041.9462


# IV. Full retraining on the whole training set

We re-encode all features on the full training set and refit the three LightGBM models  
using the optimal number of iterations found on the validation split.


In [ ]:
# Re-encoding on the full X_train
X_train_full_raw = X_train_proc[cols_features].copy()

encoder_full = OrdinalEncoder(
    cols=cat_cols_enc,
    handle_unknown="impute",
    handle_missing="impute"
)
encoder_full.fit(X_train_full_raw)

X_train_full_enc = encoder_full.transform(X_train_full_raw)
X_test_full_enc  = encoder_full.transform(X_test_raw)

# Final models with n_estimators = best_iter
lgb_freq_final = lgb.LGBMRegressor(**{**params_freq,   "n_estimators": best_iter_freq})
lgb_cm_final   = lgb.LGBMRegressor(**{**params_cm,     "n_estimators": best_iter_cm})
lgb_charge_final = lgb.LGBMRegressor(**{**params_charge, "n_estimators": best_iter_charge})

# Fit the models on the full training data
lgb_freq_final.fit(X_train_full_enc, y_freq)
lgb_cm_final.fit(X_train_full_enc, np.log1p(y_cm))
lgb_charge_final.fit(X_train_full_enc, np.log1p(y_charge))


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.253001 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4599
[LightGBM] [Info] Number of data points in the train set: 383610, number of used features: 378
[LightGBM] [Info] Start training from score -4.385889
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.222774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4602
[LightGBM] [Info] Number of data points in the train set: 383610, number of used features: 377
[LightGBM] [Info] Start training from score 0.048331
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.248365 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not 

,boosting_type,'gbdt'
,num_leaves,256
,max_depth,-1
,learning_rate,0.03
,n_estimators,29
,subsample_for_bin,200000
,objective,'regression'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,30


# V. Test predictions and competition submission file

We now generate predictions on `x_test.csv`, apply the same blending + calibration strategy,  
run final checks, and export `submission.csv` for evaluation on the ENS platform.


In [ ]:
# Predictions on test (FREQ / CM / CHARGE)
annee_test = X_test["ANNEE_ASSURANCE"].values

# FREQ
pred_freq_test = np.maximum(lgb_freq_final.predict(X_test_full_enc), 0)

# CM (log1p → CM)
pred_cm_test_log = lgb_cm_final.predict(X_test_full_enc)
pred_cm_test = np.maximum(np.expm1(pred_cm_test_log), 0)

# CHARGE multiplicative
charge_mult_test = pred_freq_test * pred_cm_test * annee_test

# CHARGE direct
pred_charge_test_log = lgb_charge_final.predict(X_test_full_enc)
charge_direct_test = np.maximum(np.expm1(pred_charge_test_log), 0)

# Apply blending + calibration
charge_final_test_raw = best_w * charge_mult_test + (1 - best_w) * charge_direct_test
charge_final_test = np.maximum(a + b * charge_final_test_raw, 0)

# Build submission DataFrame
submission = pd.DataFrame({
    "ID": X_test["ID"],
    "FREQ": pred_freq_test,
    "CM": pred_cm_test,
    "ANNEE_ASSURANCE": annee_test,
    "CHARGE": charge_final_test
})

print(submission.head())
print(submission.describe())


       ID      FREQ        CM  ANNEE_ASSURANCE      CHARGE
0  383611  0.012918  0.044802         0.813699  196.322149
1  383612  0.011436  0.039777         1.000000  186.682712
2  383613  0.011347  0.029373         0.586301  175.198321
3  383614  0.011247  0.027653         1.000000  172.797168
4  383615  0.011247  0.031730         0.753425  177.624037
                  ID          FREQ            CM  ANNEE_ASSURANCE  \
count   95852.000000  95852.000000  95852.000000     95852.000000   
mean   431536.500000      0.012401      0.050462         0.700626   
std     27670.233338      0.006437      0.050837         0.353117   
min    383611.000000      0.011048      0.012306         0.002732   
25%    407573.750000      0.011247      0.027673         0.386301   
50%    431536.500000      0.011641      0.039558         0.876712   
75%    455499.250000      0.012532      0.051440         1.000000   
max    479462.000000      0.602335      1.725034         3.000000   

             CHARGE  
co

In [ ]:
# Sanity checks
print("NaN by column:")
print(submission.isna().sum())

print("All finite:",
      np.isfinite(submission[["FREQ", "CM", "ANNEE_ASSURANCE", "CHARGE"]]).all().all())

# Difference vs naive multiplicative reconstruction
check_charge = submission["FREQ"] * submission["CM"] * submission["ANNEE_ASSURANCE"]
print("Max |CHARGE_blended - FREQ*CM*ANNEE| :",
      np.max(np.abs(check_charge - submission["CHARGE"])))


NaN by column:
ID                 0
FREQ               0
CM                 0
ANNEE_ASSURANCE    0
CHARGE             0
dtype: int64
All finite: True
Max |CHARGE_blended - FREQ*CM*ANNEE| : 2415.7875727535843


In [ ]:
# Export CSV
submission.to_csv("submission.csv", index=False)
print(">> submission.csv created")


>> submission.csv created
